In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
import pandas_datareader as pdr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import arrow

In [ ]:
ticker = pdr.quandl.QuandlReader('AAPL', '2000-01-01', '2018-03-27').read()

In [ ]:
ticker = ticker.sort_index()

In [ ]:
ticker.loc['2007'].head()

In [ ]:
ticker.index

In [ ]:
ticker.columns

In [ ]:
ts = ticker['Close'][-10:]

In [ ]:
type(ts)

In [ ]:
ticker.loc[pd.Timestamp('2006-11-01'):pd.Timestamp('2006-12-31')].head()

In [ ]:
monthly = ticker.resample('M').mean()

In [ ]:
print(monthly.tail())

In [ ]:
ticker.asfreq('M', method='bfill')

In [ ]:
ticker['Close'].plot(grid=True)

In [ ]:
daily_close = ticker[['AdjClose']]

In [ ]:
daily_pct_change = daily_close.pct_change()

In [ ]:
# Assign `Adj Close` to `daily_close`
daily_close = ticker[['AdjClose']]

# Daily returns
daily_pct_change = daily_close.pct_change()

# Replace NA values with 0
daily_pct_change.fillna(0, inplace=True)

# Inspect daily returns
print(daily_pct_change)

# Daily log returns
daily_log_returns = np.log(daily_close.pct_change()+1)

# Print daily log returns
print(daily_log_returns)

In [ ]:
# Resample `ticker` to business months, take last observation as value 
monthly = ticker.resample('BM').apply(lambda x: x[-1])

# calculate monthly percentage change
print(monthly.pct_change())

# resample ticker to quarters, take the mean as value per quarter
quarter = ticker.resample('4M').mean()

#quarter.pct_change()


In [ ]:
# this is what pct_change does behind the scenes
daily_pct_change = daily_close / daily_close.shift(1) - 1

# Print `daily_pct_change`
print(daily_pct_change)

In [ ]:
daily_log_returns_shift = np.log(daily_close / daily_close.shift(1))

In [ ]:
print(daily_log_returns_shift)

In [ ]:
# Plot the distribution of `daily_pct_c`
daily_pct_change.hist(bins=50)

# Show the plot
plt.show()

# Pull up summary statistics
print(daily_pct_change.describe())

In [ ]:
# Calculate the cumulative daily returns
cum_daily_return = (1 + daily_pct_change).cumprod()

# Print `cum_daily_return`
print(cum_daily_return)

In [ ]:
# Plot the cumulative daily returns
cum_daily_return.plot(figsize=(12,8))

# Show the plot
plt.show()

In [ ]:
cum_monthly_return = cum_daily_return.resample('M').mean()
print(cum_monthly_return)

In [ ]:
daily_returns = ticker['AdjClose'].pct_change()


In [ ]:
print(daily_returns)

In [ ]:
monthly_close = ticker['AdjClose'].resample('M').ffill()

In [ ]:
monthly_close

In [ ]:
now = arrow.get('2017-10-03')

In [ ]:
# calculate lookback period for momentum
# start period defined as the first day of the month, 13 months ago, enabling us to get initial value to compare
start_period = now.shift(months=-12).replace(day=1).format('YYYY-MM-DD')
# end period excludes the most recent full month, (exclusive)
end_period = now.shift(months=-1).replace(day=1).format('YYYY-MM-DD')

In [ ]:
print(start_period)
print(end_period)

In [ ]:
# calculate percent change taking the last value as observation
monthly_ticker = ticker[start_period:end_period].resample('M').apply(lambda x: x[-1]).pct_change()

In [ ]:
monthly_ticker["gross_returns"] = monthly_ticker["AdjClose"] + 1

In [ ]:
ticker[start_period:end_period]

In [ ]:
ticker['daily_returns'] = ticker[start_period:end_period]['AdjClose'].pct_change()

In [ ]:
monthly_returns = ticker[start_period:end_period]['daily_returns'].resample('M').apply(lambda x: ((x + 1).cumprod()).last("D"))

In [ ]:
monthly_returns

## How to calculate "momentum" according to Quantitative Momentum
> To calculate the cumulative return over the past 12 months, we take the net return streams from each month and turn them into gross returns by adding 1. Thus, if Apple's net returns for January are –10.77 percent, Apple's gross returns for January are 0.8923 (–0.1077 + 1). Then, we multiply all the gross return series (i.e., months) and subtract 1 to find the cumulative 12-month net return. ([Location 2467](https://readwise.io/to_kindle?action=open&asin=B01LY6P2LB&location=2467))

In [ ]:
twelve_month_lookback_momentum = np.prod(monthly_returns) - 1
print(twelve_month_lookback_momentum)